In [1]:
// general includes
#include <iostream>   // std::cout|endl 
#include <memory>   // std::unique_ptr
#include <cstdio>  // std::fopen|fprintf|fclose
struct Widget {
    int m;
};

## Quick Recap

...

## Goals for today

...

## Smart Pointers

Raw/native pointers are a versatile tool in C. 
They are used for:

- (a) passing an object (reference) for inspection purposes
  ```cpp
  void func(const Widget * const w){
      int m = w->m; // allowed ??
      w->m = 5;     // allowed ?? 
      ++w;          // allowed ??
      if (w == NULL) return; // allowed ??
  }
  ``` 
- (b) passing an object (reference) to be modified by callee
  ```cpp
  void func(Widget * const w){
      int m = w->m; // allowed ??
      w->m = 5;     // allowed ?? 
      ++w;          // allowed ??
      if (w == NULL) return; // allowed ??
  }
  ```   
- (c) passing an array object (reference)
  ```cpp
  void func(const Widget * array, int n){  
  void func(const Widget[] array, int n){
      int m0 = w[0].m;   // allowed ??
      int mn = w[n-1].m; // allowed ??      
      w[0].m = 5;     // allowed ?? 
      ++w;          // allowed ??
      if (w == NULL) return; // allowed ??
  }
  ```    
- (d) returning a reference for inspecting/modify purposes 
  ```cpp
  Widget * find(Widget * array, int n ){  
    ...
  }  
  ```
- (e) returning a reference (and ownership?) to a dynamic object 
  ```cpp
  Widget * create(){  
    ...
  }  
  ```   

In the last two examples above it becomes aparent that that *ownership* is hard to express in C only using *raw pointers*. Let's revisit the above examples in C++ using *references* and `std::unique_ptr` to express ownership and expectations:


- (a) passing an object (reference) for inspection purposes
  ```cpp
  void func(const Widget &w){
      int m = w->m; // allowed ??
      w->m = 5;     // allowed ?? 
      ++w;          // allowed ??
      if (w == NULL) return; // allowed ??
  }
  ``` 
- (b) passing an object (reference) to be modified by callee
  ```cpp
  void func(Widget &w){
      int m = w->m; // allowed ??
      w->m = 5;     // allowed ?? 
      ++w;          // allowed ??
      if (w == NULL) return; // allowed ??
  }
  ```   
- (c) passing an array object (reference)
  ```cpp
  void func(const std::array<Widget,N> & array){    
  void func(const std::vector<Widget> & array){  
      int m0 = array[0].m;   // allowed ??
      int mn = array[n-1].m; // allowed ??      
      array[0].m = 5;     // allowed ?? 
      ++array;          // allowed ??
      if (array == NULL) return; // allowed ??
  }
  ```    
- (d) returning a reference for inspecting/modify purposes 
  ```cpp
  Widget & find(const std::vector<Widget> & array){  
    ...
  }  
  ```
- (e1) returning a reference **and** ownership to a dynamic object 
  ```cpp
  ??? create(){  
    ...
  }  
- (e2) returning a pointer (allowing arithmetics/nullable) but **no** ownership to an object 
  ```cpp
  ??? func(){  
    ...
  }    
  ```   

**What options does C++ provide to enable clear ownership sematics for dynamic resources?**
- `std::unique_ptr` [(cppref)](https://en.cppreference.com/w/cpp/memory/unique_ptr): exclusive ownership, "owner turns of the lights"  
- `std::shared_ptr` [(cppref)](https://en.cppreference.com/w/cpp/memory/shared_ptr): shared ownership, "last one turns off the lights" --> *reference counting*

As the smart pointers in the C++ standard library [(cppref)](https://en.cppreference.com/w/cpp/memory) aim to be used for functions/interfaces instead of *raw pointers* let's first list some functionality we expect to work for a *raw pointer*:

In [17]:
struct Widget{
    int m;
};

Widget w; 
Widget *ptr = &w;                     // (1) init/construction
Widget *ptr2 = ptr;                   // (2) init/construction
ptr2 = &w;                            // (3) assign
ptr ->m = 5;                          // (4) member access using ->
( *ptr).m = 6;                        // (5) dereferencing + .
 ++ptr;                               // (6) increment
 --ptr;                               // (7) decrement
if (ptr  != &w) {throw;};             // (8) operator!= 
ptr  = nullptr;                       // (9) nullable
if (ptr  == &w) {throw;};             // (10) operator==
Widget *ptr3  = new Widget{};         // (11) handle to a dynamic resource
Widget *array = new Widget[3]{w,w,w}; // (12a) dynamic arrays
array[2]= Widget{2};                  // (12b) indexed array access

@0x560f8dea4548

## std::unique_ptr

The class `std::unique_ptr` aims to follow the functionality/semantics  for raw pointers as long as it does not stand against the underlying idea:

>  In cases where an exclusive ownership of a resource is desired, `std::unique_ptr` provides (and demands) expressiveness and delivers some convenience.

Let's consider a function `get_widget`, which returns a raw pointer to a dynamically allocated `Widget`:
```cpp
Widget *ptr = get_widget();
// is ptr a resource which needs to be be released?
// who will release the resource?
```

Let's transform `get_widget` to construct and return a `std::unique_ptr` (which expresses that the caller is the new owner):

In [3]:
std::unique_ptr<Widget> get_widget() {

  // (1) passing pointer directly (in return expression)
  return std::unique_ptr<Widget>(new Widget{});

  // // (2) ... via make_unique (in return expression)
  return std::make_unique<Widget>(); 

  // // (3) create object, rely on implicit conversions (due to return type of function)
  auto tmp = new Widget{};
  return std::unique_ptr<Widget>(tmp); // implicit construction of unique_ptr

  // // (4) create directly and return
  std::unique_ptr<Widget> up(new Widget{}); // currently myfunc owns it
  return std::move(up);   

}

auto ptr = get_widget(); 


**What do we know about the return value `ptr` of `get_widget` above?**
- we obtained a dynamic resource from the function that we own now.
- the resource is wrapped in a unique_ptr: can be move but not copied.
- we do not have to deallocate/free; the unique_ptr-destructor will do this


**What happens if the return value is not captured in line (5) above, is this a leak?**
- no problem: destructor will clean up.

**Why do we even use pointers in C++ and not only references?**
- reference cannot be reassigned,e.g. swapping two references means you would have to re-assign.
- a reference is cannot be NULL, e.g. if you want to use NULL in your interface to encode certain situations, this cannot be achieved returning a reference.
- a reference cannot transfer ownership of a dynamic resource (there is no way to `delete` via a reference).
- a reference does not allow to perform pointer arithmetics.
- BUT: if non of the above constraints is relevant: use a reference instead of a pointer!


Now the return type is expressive (it is clear the caller becomes the owner of the returned object) and access works the same way as for a raw pointer:


In [7]:
std::unique_ptr<Widget> smart = get_widget();
smart->m = 5;     // same as for a raw pointer
(*smart).m = 10;  // same as for a raw pointer
// no manual termination of lifetime required:
// destruction happens when local variable 'smart' goes out of scope

10

As we can see, the returned object is usable like a raw pointer w.r.t. to accessing the pointed-to object.
Additionally, manual termination is not required: the owned resource is released when the smart pointer variable goes out of scope.



**Which further restrictions (SMFs) can we expect if the `std::unique_ptr` wants to model a **unique** handle to a resource?**

```cpp
std::unique_ptr<Widget> w3 = get_widget();  // ctor ?
std::unique_ptr<Widget> w2 = get_widget();  // ctor ?
w2 = w3;                                    // (1) copy assign ?
std::unique_ptr<Widget> w4 = w2;            // (2) copy ctor ?
std::unique_ptr<Widget> w3 = std::move(w2); // (3) move ctor ? 
w2 = std::move(w3));                        // (4) move assign?
```

**In which of the SMFs of `unique_ptr` is a release of the resource required?**
- in the destructor
- during move assignment (if a resource is present).

**What about pointer arithmetic and other operators? Are they supported?**
- arithmetics, e.g., `operator++`, `operator--` ?? 
  - not available (but of course you can do that: `.get()` the raw pointer)
- comparison, e.g., `opterator==`, `opterator!=` ??   
  - available, we definitely want a comparison to `nullptr`

### Overhead: what does a `std::unique_ptr` look like?

If we decide to use a `std::unique_ptr` instead of a raw pointer, what can we expect in terms of performance? 
To argue about this, let's look what (a small part of) a simplified implementation looks like:


```cpp
template <typename T> class unique_ptr {
  T *ptr = nullptr;
public:
  unique_ptr(T *p) : ptr(p) {}
  ~unique_ptr() { delete ptr; }
  T &operator*() { return *ptr; } 
  T *operator->() { return ptr; }
  ...
};
template <typename T, typename... ARGS>
unique_ptr<T> make_unique(ARGS&&... args) {
  return unique_ptr<T>(new T(std::forward<ARGS>(args)...));
}
```

We can see that it is a thin wrapper class with a single raw pointer member `ptr` which is initialized from the passed pointer. For this we can expect the memory overhead to be zero. 
Construction/access/destruction have minimal overhead (expected to be negligible when compiling with optimization).When passing to a function (compared to a raw pointer parameter), there is minimal overhead due to ABI contraints [(link)](https://www.youtube.com/watch?v=rHIkrotSwcc)

### Custom deleter example

If a `std::unique_ptr` manages a resource which is not a dynamically allocated object (for which the default release mechanism is applicable, i.e., `delete` or `delete[]`) a *custom deleter* can be passed at construction. 
Let's look at a small example which uses a file handle:

In [2]:
void parse(FILE *handle){ /* parsing a file */ } 

In [3]:
auto filename = "data.json";
auto mode = "r";
FILE *handle = std::fopen(filename, mode); // obtain resource, i.e., a custom 'new'
if (handle) {
  parse(handle);
  std::fclose(handle); // release resource, i.e., a custom 'delete'
}


If we would like to wrap the file handle obtained from `fopen` using a `std::unique_ptr` the default release mechanism `delete` does not apply but instead `fclose` is required to be called on the handle. This can be achieved by using a custom function object as deleter:

In [4]:
auto open(const char *name, const char *mode) {
  auto deleter = [](FILE *handle) { std::fclose(handle); };
  return std::unique_ptr<FILE, decltype(deleter)>(std::fopen(name, mode), deleter);
};

In [5]:
auto filename = "data.json";
auto mode = "r";
auto file = open(filename, mode);
if (file) { // (1)
  parse(file.get()); // (2) 
}

**Is there an implicit conversion from `std::unique_ptr` to the underlying raw pointer at line (2) above ? What about (1) ?**
- (2): no, one as to rely on `.get()` [(cppref)](https://en.cppreference.com/w/cpp/memory/unique_ptr/get):
  ```cpp
      parse(file.get()); // use functionality of unique_ptr
      parse(&*file); // more cumbersome (this is why .get() is there)
  ```
- (1): works, the conversion function (`operator bool`) is implemented for convenience [(cppref)](https://en.cppreference.com/w/cpp/memory/unique_ptr/operator_bool).

This demonstrates that a `std::unique_ptr` can also be used to manage resources which are not (directly) created with `new` and released with `delete`.

#### Managing an array
When a `std::unique_ptr` is used to manage an array, indexed access is provided via the `[]` operator:

```cpp
std::unique_ptr<double[]> up(new double [10]);
up[9] = ...; // set last element of array
```

The relase of the array (which now requires `delete[]`) is realized by an appropriate deleter in the template specialization of `std::unique_ptr`, see second (2) template class declaration here: [(cppref)](https://en.cppreference.com/w/cpp/memory/unique_ptr)

## Summary `std::unique_ptr`

- indicates exclusive ownership.
- try to make pointers safer, but keep semantics:
  - safe: deleters are automatically called at end of lifetime.
  - semantics: interface/semantics tries to imitate raw pointer semantics whenever possible.
  - reassiging: releases old resource (including deletion).
- only one handle per resource at a time (unique) is allowed:
  - disabled copy construction
  - disabled copy assignment.
- lightweight (no memory footprint overhead).
- custom deleter available.
